In [6]:
from dotenv import load_dotenv

load_dotenv()

True

In [29]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser, MarkdownListOutputParser

llm = ChatOpenAI()

## Especialista em direito

In [40]:
TEMPLATE = """
Você é um especialista jurídico e professor de direito no Brasil
Auxilia os alunos na criação de mapa mental (MindMap)
O mapa mental deve conter a ideia principal, ramos e subramos, sempre apontar os artigos 
Context:{input}
Formate a saida do mapa mental em mardown 
"""


In [41]:
prompt = ChatPromptTemplate.from_template(template=TEMPLATE)
output_parser = StrOutputParser()
chain = prompt | llm | output_parser


In [42]:
res = chain.invoke({"input": "liste os diretiros fundamentais na constituição brasileira"})

In [43]:
print(res)

# Direitos Fundamentais na Constituição Brasileira

- **I - Direitos Individuais e Coletivos:**
  - **Artigos 5º ao 17**
    - Direito à vida
    - Direito à liberdade
    - Direito à igualdade
    - Direito à segurança
    - Direito à propriedade
    - Direito à intimidade
    - Direito à liberdade de expressão
    - Direito à liberdade de religião
    - Direito à liberdade de associação
    - Direito à liberdade de reunião
    - Direito à privacidade
    - Direito à inviolabilidade do domicílio

- **II - Direitos Sociais:**
  - **Artigos 6º ao 11**
    - Direito à educação
    - Direito à saúde
    - Direito ao trabalho
    - Direito à moradia
    - Direito à segurança
    - Direito à previdência social

- **III - Direitos de Nacionalidade:**
  - **Artigos 12 ao 16**
    - Direito à nacionalidade
    - Direito à naturalização
    - Direito à perda da nacionalidade

- **IV - Direitos Políticos:**
  - **Artigos 14 ao 17**
    - Direito de votar
    - Direito de ser votado
    - Direito